##### Spoiler Alert! Spoiler Detection Project

## Combine text and metadata (Sentence-wise, balanced RF Classifier)

In [1]:
#Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.utils import class_weighter
from imblearn.ensemble import BalancedRandomForestClassifier

#### Load Data

In [2]:
train = pd.read_json('data/train_reduced.json')
train.drop(['user_id', 'title'], inplace = True, axis = 1)
#Correct the review_length column
train['review_len'] = train['tokenized'].apply(lambda x: len(x))

In [3]:
val = pd.read_json('data/validation_reduced.json')
val.drop(['user_id', 'title'], inplace = True, axis = 1)
#Correct the review_length column
val['review_len'] = val['tokenized'].apply(lambda x: len(x))

In [4]:
X_tr = pd.read_csv('data/X_train.csv')
X_v = pd.read_csv('data/X_val.csv')
X_tr.rename(columns = {"0": 'review_whole'}, inplace = True)
X_v.rename(columns = {"0": 'review_whole'}, inplace = True)

In [5]:
train = pd.concat([train, X_tr.review_whole], axis = 1)
val = pd.concat([val, X_v.review_whole], axis = 1)

In [6]:
train.to_json('data/train_only_needed_columns.json')
val.to_json('data/val_only_needed_columns.json')

In [7]:
#Since the data is very inbalanced, we only take the training reviews containing spoiler sentences. 
train = train[train['spoiler_dum']== 1]

In [9]:
train.reset_index(inplace = True, drop = True)

In [12]:
def get_sentencewise_df(df):
    
    '''Get review sentences from a dataframe df given in.
    The review sentences are written in a list 'lst' which is then transformed
    into a numpy array'''
    
    sen_lst = []
    genre_lst = []
    len_lst = []
    label_lst = []
    for i in range(len(df)):
        for sentence in df['tokenized'][i]:
            sen_lst.append(sentence)
            genre_lst.append(df['genre'][i])
            len_lst.append(df['review_len'][i])
            
    for labellist in df['sentence_labels']:
        for label in labellist:
            label_lst.append(label)
    df = pd.concat([pd.Series(sen_lst), pd.Series(label_lst), pd.Series(genre_lst), pd.Series(len_lst)], axis = 1)
    return df

In [13]:
train = get_sentencewise_df(train)
train.rename(columns = {0: 'sentence', 1: 'spoiler_dum', 2: 'genre', 3: 'review_length'}, inplace = True)
train.head()

,sentence,spoiler_dum,genre,review_length
0,overall plot book get star maybe,0,"fantasy, paranormal",11
1,bad trilogy end,0,"fantasy, paranormal",11
2,feeling chase character aside basically feelin...,1,"fantasy, paranormal",11
3,unacceptable,1,"fantasy, paranormal",11
4,give book star go able read case later book se...,0,"fantasy, paranormal",11


In [14]:
val = get_sentencewise_df(val)
val.rename(columns = {0: 'sentence', 1: 'spoiler_dum', 2: 'genre', 3: 'review_length'}, inplace = True)
val.head()

,sentence,spoiler_dum,genre,review_length
0,look like promising new fantasy series,0,"fantasy, paranormal",3
1,love character definitely omg moment read,0,"fantasy, paranormal",3
2,negative wish term mythology explain little well,0,"fantasy, paranormal",3
3,definitely remind inuyasha read,0,"fantasy, paranormal",7
4,blend humour romance delightful,0,"fantasy, paranormal",7


In [15]:
X_train = train[['sentence', 'genre', 'review_length']]
X_train_rev = train.sentence
y_train = train.spoiler_dum

In [16]:
X_val = val[['sentence', 'genre', 'review_length']]
X_val_rev = val.sentence
y_val = val.spoiler_dum

#### Balanced Random Forest

In [21]:
#Build a pipeline for feature extraction with TF IDF and SGD
#TFIDF
tfidf = TfidfVectorizer(stop_words = 'english', ngram_range = (1,1), min_df = 100)

#Feature Selection: K-Best
feat = SelectKBest(score_func = chi2, k = 5000)

#Balanced Random Forest classifier
brf = BalancedRandomForestClassifier(n_estimators = 100, random_state = 42, verbose = 5, n_jobs = -1, class_weight = 'balanced', oob_score = True)

pipe = Pipeline([('TFIDF', tfidf),('select_kbest', feat), ('BRF', brf)])

In [22]:
#Function for model training and prediction
def run_model(pipeline, X_train, y_train, X_test, y_test):
    #Fit the model
    model = pipeline.fit(X_train, y_train)
    
    #Predict labels of test data
    y_pred = model.predict(X_test)
    
    #Print classification report and confusion matrix
    return print(classification_report(y_test, y_pred)), print(confusion_matrix(y_test, y_pred, normalize = 'true'))
    return y_pred

In [23]:
#Run the model sentence-wise (validation data)
run_model(pipe, X_train_rev, y_train, X_val_rev, y_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 3 of 100
building tree 2 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 20.2min


building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 106.8min


building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 165.9min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  9.3min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed: 14.5min finished


              precision    recall  f1-score   support

           0       1.00      0.57      0.72   3396530
           1       0.07      0.92      0.12    113966

    accuracy                           0.58   3510496
   macro avg       0.53      0.74      0.42   3510496
weighted avg       0.96      0.58      0.70   3510496

[[0.56551157 0.43448843]
 [0.08260358 0.91739642]]


(None, None)